# Deliverable 2
__Names and Uid__  
Albert Perez Paredes U214020  
Pol Garcia Lopez U214574

### Imports

In [1]:
import nltk
import time
import numpy as np
nltk.download('stopwords')

import sys
from pathlib import Path

[nltk_data] Downloading package stopwords to /home/pol/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Loading the json

In [2]:
document_path = r"../../data/fashion_products_dataset.json"
# sys
# pathlib Path

project_root = Path().resolve().parent.parent
sys.path.append(str(project_root))
import importlib

from myapp.search.load_corpus import load_corpus
docs = load_corpus(document_path)


from myapp.search.algorithms import create_index_tfidf, create_index_tfidf_log, compute_line_docs
from myapp.search.algorithms import search_tfidf
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
import seaborn as sns

#Pt2
from myapp.search.algorithms import precision_at_K, select_queries

import myapp.search.algorithms as alg

### Implementation

__Preprocessing documents__

In [3]:
#myapp.search.algorithms create_index_tfidf, compute_line_docs

start_time = time.time()
docs = compute_line_docs(docs)
print("Total time to preprocess documents: {} seconds" .format(np.round(time.time() - start_time, 2)))

Total time to preprocess documents: 55.81 seconds


# __Part 1__ Indexing

__Index_tfidf__  

In [4]:
start_time = time.time()
num_documents = len(docs)
index, tf, df, idf, title_index = create_index_tfidf(docs, num_documents)
print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time, 2)))

Total time to create the index: 30.11 seconds


__Querying__  


In [ ]:
# myapp.search.algorithms search tfidf
print("Insert your query (i.e.: women full sleeve sweatshirt cotton):\n")
query = input()
# query = "women full sleeve sweatshirt cotton"
print("\nSearching for: {}\n".format(query)) #Añadido para que en Visual Studio Code se vea la query introducida
ranked_docs = search_tfidf(query, index, idf, tf, title_index)
top = 30

print("\n======================\nTop {} results out of {} for the searched query:\n".format(top, len(ranked_docs)))
for d_id in ranked_docs[:top]:
    print("page_id= {} - page_title: {}".format(d_id, title_index[d_id]))

Insert your query (i.e.: women full sleeve sweatshirt cotton):



# __Part 2__ Evaluation

In [ ]:
queries_real = pd.read_csv(r"../../data/validation_labels.csv")
queries_real.head(10) #solo hay 40 de momento

,title,pid,query_id,labels
0,Full Sleeve Printed Women Sweatshirt,SWSFFVKBCQG5FHPF,1,1
1,Full Sleeve Striped Women Sweatshirt,SWSFJY5ZFHQ7HXKW,1,0
2,Full Sleeve Printed Women Sweatshirt,SWSFUY89NHMZHZPX,1,1
3,Full Sleeve Graphic Print Women Sweatshirt,SWSFXQ5YX6RZKHP4,1,1
4,Full Sleeve Solid Women Sweatshirt,JCKFTZBC3DMCVYXH,1,0
5,Full Sleeve Graphic Print Women Sweatshirt,SWSFWCTDHRABJFCR,1,1
6,Typography Women V Neck Multicolor T-Shirt (P...,TSHFUH9FABXHSDPJ,1,0
7,Full Sleeve Printed Women Sweatshirt,SWSFVGKENHBSVUWJ,1,1
8,Full Sleeve Solid Women Sweatshirt,JCKF7M8DNBB6WZ8Y,1,1
9,Full Sleeve Graphic Print Women Sweatshirt,SWSFWCSPEGMDH8FV,1,1


__Pt2.2__

In [ ]:
importlib.reload(alg)
from myapp.search.algorithms import create_index_tfidf_log

start_time = time.time()
num_documents = len(docs)
index, tf, df, idf, title_index = create_index_tfidf(docs, num_documents)
index_log, tf_log, df, idf, title_index = create_index_tfidf_log(docs, num_documents)
print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time, 2)))

Precision

In [ ]:
from myapp.search.algorithms import precision_at_K

k = 277 # elemento 51 es relevant # elemento 274 tambien # elemento 277 también
queries = ["Full Sleeve Printed Women Sweatshirt"]
queries_sel = select_queries(queries, queries_real)

for i, q in enumerate(queries_sel):
    ranked_docs_t = search_tfidf(queries[i], index, idf, tf, title_index)
    precision_t = precision_at_K(q, ranked_docs_t, k)
    print("The query {%s} has precision %0.3f at k=%d" % (queries[i], precision_t, k))

The query {Full Sleeve Printed Women Sweatshirt} has precision 0.011 at k=277


Recall

In [ ]:
importlib.reload(alg)
from myapp.search.algorithms import recall_at_K

k = 200 # elemento 51 es relevant # elemento 274 tambien # elemento 277 también
queries = ["Full Sleeve Printed Women Sweatshirt"]
queries_sel = select_queries(queries, queries_real)

for i, q in enumerate(queries_sel):
    ranked_docs_t = search_tfidf(queries[i], index, idf, tf, title_index)
    recall_t = recall_at_K(q, ranked_docs_t, k)
    print("The query {%s} has recall %0.3f at k=%d" % (queries[i], recall_t, k))

The query {Full Sleeve Printed Women Sweatshirt} has recall 0.333 at k=200


Average Precision

In [ ]:
importlib.reload(alg)
from myapp.search.algorithms import average_precision

# elemento 51 es relevant # elemento 274 tambien # elemento 277 también
queries = ["Full Sleeve Printed Women Sweatshirt"]
queries_sel = select_queries(queries, queries_real)

for i, q in enumerate(queries_sel):
    ranked_docs_t = search_tfidf(queries[i], index, idf, tf, title_index)
    avgprecision_t = average_precision(q, ranked_docs_t)
    print("The query {%s} has average precision %0.3f" % (queries[i], avgprecision_t))

The query {Full Sleeve Printed Women Sweatshirt} has average precision 0.013


F1_score

In [ ]:
importlib.reload(alg)
from myapp.search.algorithms import f1_score_at_K, f1_score_at_K_optimized

k = 1000 # elemento 51 es relevant # elemento 274 tambien # elemento 277 también
queries = ["Full Sleeve Printed Women Sweatshirt"]
queries_sel = select_queries(queries, queries_real)

for i, q in enumerate(queries_sel):
    ranked_docs_t = search_tfidf(queries[i], index, idf, tf, title_index)
    f1_score = f1_score_at_K(q, ranked_docs_t, k)
    f1_score_opt = f1_score_at_K_optimized(q, ranked_docs_t, k)
    print("The query {%s} has F1-score %0.3f vs Opt %0.3f at k=%d" % (queries[i], f1_score, f1_score_opt, k))

The query {Full Sleeve Printed Women Sweatshirt} has F1-score 0.006 vs Opt 0.006 at k=1000


MAP

In [ ]:
importlib.reload(alg)
from myapp.search.algorithms import mean_average_precision
from myapp.search.algorithms import average_precision


queries_only = ["Full Sleeve Printed Women Sweatshirt","Typography Women V Neck Multicolor T-Shirt  (Pack of 2)"]
queries_sel = select_queries(queries, queries_real)

map_map = mean_average_precision(queries_real, index, idf, tf, title_index)
map_log = mean_average_precision(queries_real, index, idf, tf, title_index, log = 1)
print("The ranking for these queries has mean average precision %0.3f , %0.3f" % (map_map, map_log))

The ranking for these queries has mean average precision 0.094


MRR

In [ ]:
importlib.reload(alg)
from myapp.search.algorithms import mean_reciprocal_rank


queries_only = ["Full Sleeve Printed Women Sweatshirt","Typography Women V Neck Multicolor T-Shirt  (Pack of 2)"]
queries_sel = select_queries(queries, queries_real)


mrr = mean_reciprocal_rank(queries_real, index, idf, tf, title_index, 250)
mrr = mean_reciprocal_rank(queries_real, index, idf, tf, title_index, 250, log = 1)
print("The ranking for these queries has MRR  %0.3f, %0.3f" % (mrr, mrr_log))

The ranking for these queries has MRR  0.149
